In [2]:


# ========================
# CAN + UDS Setup
# ========================

# Define CAN IDs for Diagnostics
CAN_IDS = {
    "Tester_Request": 0x7DF,   # Functional request (broadcast)
    "ECU_Response": 0x7E8      # ECU replies here
}

# Define a stored Diagnostic Trouble Code in ECU
STORED_DTC = "P0300"   # Example: Random/Multiple Cylinder Misfire

# ECU database for UDS requests/responses (simulated)
ECU_RESPONSES = {
    "0x19 02": f"0x59 02 {STORED_DTC}",  # Read DTC → Misfire
    "0x14": "0x54",                      # Clear DTC → Positive Response
    "0x11 01": "0x51 01",                # ECU Reset → Positive Response
    "INVALID": "0x7F"                    # Negative Response
}


# ========================
# CAN + UDS Demo Functions
# ========================

def tester_send_request():
    """Simulate tester sending a ReadDTC request (0x19 02)."""
    print(f"Tester -> ECU: Request DTC (CAN ID: {hex(CAN_IDS['Tester_Request'])})")
    uds_request = {
        "ID": CAN_IDS["Tester_Request"],
        "Data": "02 19 02 00 00 00 00 00"  # UDS request: Read DTC
    }
    return uds_request

def ecu_send_response():
    """Simulate ECU sending a response with stored DTC."""
    print(f"ECU -> Tester: Reply with stored DTC (CAN ID: {hex(CAN_IDS['ECU_Response'])})")
    uds_response = {
        "ID": CAN_IDS["ECU_Response"],
        "Data": "06 59 02 01 03 00 00 00",  # Encoded DTC
        "Decoded_DTC": STORED_DTC
    }
    return uds_response


# ========================
# Test Framework Functions
# ========================

def send_request(request):
    """Generic function to send request to ECU and get response."""
    print(f"Tester → ECU : {request}")
    response = ECU_RESPONSES.get(request, ECU_RESPONSES["INVALID"])
    print(f"ECU → Tester : {response}\n")
    return response


# ------------------------
# Test Cases
# ------------------------

def test_read_dtc():
    print("TC_01: Read DTC")
    response = send_request("0x19 02")
    assert response.startswith("0x59"), "FAIL: Wrong Service ID"
    print("PASS ✅ Read DTC\n")

def test_clear_dtc():
    print("TC_02: Clear DTC")
    response = send_request("0x14")
    assert response == "0x54", "FAIL: DTC not cleared"
    print("PASS ✅ Clear DTC\n")

def test_ecu_reset():
    print("TC_03: ECU Reset")
    response = send_request("0x11 01")
    assert response == "0x51 01", "FAIL: ECU did not reset"
    print("PASS ✅ ECU Reset\n")

def test_negative_response():
    print("TC_04: Negative Response")
    response = send_request("0x55")  # Invalid request
    assert response.startswith("0x7F"), "FAIL: ECU did not send negative response"
    print("PASS ✅ Negative Response\n")


# ========================
# Main Simulation
# ========================

if __name__ == "__main__":
    print("\n🚗 UDS Diagnostic Simulation Started...\n")

    # --- Demo Part (Project 333) ---
    print("🔹 DEMO: CAN + UDS Request/Response")
    request = tester_send_request()
    print("Request Frame:", request)
    response = ecu_send_response()
    print("Response Frame:", response)
    print(f"\n✅ Tester received DTC: {response['Decoded_DTC']} (Random Misfire Detected)\n")

    # --- Test Framework Part (Project 4444) ---
    print("🔹 TEST CASES: UDS Validation\n")
    test_read_dtc()
    test_clear_dtc()
    test_ecu_reset()
    test_negative_response()

    print("🎉 All demo + test cases executed successfully!")



🚗 UDS Diagnostic Simulation Started...

🔹 DEMO: CAN + UDS Request/Response
Tester -> ECU: Request DTC (CAN ID: 0x7df)
Request Frame: {'ID': 2015, 'Data': '02 19 02 00 00 00 00 00'}
ECU -> Tester: Reply with stored DTC (CAN ID: 0x7e8)
Response Frame: {'ID': 2024, 'Data': '06 59 02 01 03 00 00 00', 'Decoded_DTC': 'P0300'}

✅ Tester received DTC: P0300 (Random Misfire Detected)

🔹 TEST CASES: UDS Validation

TC_01: Read DTC
Tester → ECU : 0x19 02
ECU → Tester : 0x59 02 P0300

PASS ✅ Read DTC

TC_02: Clear DTC
Tester → ECU : 0x14
ECU → Tester : 0x54

PASS ✅ Clear DTC

TC_03: ECU Reset
Tester → ECU : 0x11 01
ECU → Tester : 0x51 01

PASS ✅ ECU Reset

TC_04: Negative Response
Tester → ECU : 0x55
ECU → Tester : 0x7F

PASS ✅ Negative Response

🎉 All demo + test cases executed successfully!
